In [ ]:
def clamp(low, value, high):
    return min(max(low, value), high)

def new_bit_board(rows=6, cols=7, sequence=4, gravity=True, turn=0):
    rows_b = clamp(0, rows, 15) << 0 # 4 bits
    cols_b = clamp(0, cols, 15) << 4 # 4 bits
    sequence_b = clamp(0, sequence, 15) << 8 # 4 bits
    gravity_b = (gravity & 1) << 12 # 1 bit
    turn_b = (turn & 1) << 13 # 1 bit
    return rows_b | cols_b | sequence_b | gravity_b | turn_b

def get_sub_bits(bits, n_bits, start):
    return (bits & (((1 << n_bits) - 1) << start)) >> start

def get_rows(bit_board): return get_sub_bits(bit_board, 4, 0)
def get_cols(bit_board): return get_sub_bits(bit_board, 4, 4)
def get_sequence(bit_board): return get_sub_bits(bit_board, 4, 8)
def get_gravity(bit_board): return get_sub_bits(bit_board, 1, 12)
def get_turn(bit_board): return get_sub_bits(bit_board, 1, 13)
def get_board(bit_board, player=0):
    rows_cols = get_rows(bit_board) * get_cols(bit_board)
    return get_sub_bits(bit_board, rows_cols, 14 + rows_cols * player)

def get_bits(bit_board, cell_index, mode="row"):
    modes = ["row", "col", "diag_1", "diag_2"]
    if mode not in modes: raise ValueError(f'Invalid mode: {mode}')

    rows, cols = get_rows(bit_board), get_cols(bit_board)
    sequence = get_sequence(bit_board) - 1
    boards = [get_board(bit_board, player) for player in [0, 1]]

    row_i, col_i = cell_index
    row_t = 1 - (mode == "row")
    col_t = dict(zip(modes, [1, 0, 1, -1]))[mode]

    row_min = max(0, row_i - sequence * row_t) - row_i
    col_min = max(0, col_i - sequence * col_t) - col_i
    range_min = max(row_min, col_min)
    row_max = min(row_i + sequence * row_t, rows) - row_i
    col_max = min(col_i + sequence * col_t, cols) - col_i
    range_max = min(row_max, col_max)

    return [
        sum(
            1 << i
            for i, n in enumerate(range(range_min, range_max))
            if board & (1 << (row_i + row_t * n) * cols + (col_i + col_t * n))
        )
        for board in boards
    ]

# TODO: left off here

In [10]:
RULES = {
    "tic-tac-toe": {
        "rows": 3,
        "cols": 3,
        "sequence": 3,
        "gravity": False,
    },
    "connect-4": {
        "rows": 6,
        "cols": 7,
        "sequence": 4,
        "gravity": True,
    }
}

bit_board = new_bit_board(**RULES["tic-tac-toe"])
bin(bit_board)

'0b1100110011'

In [ ]:
def flip_bit_board(bit_board, direction="horizontal"):

    if direction == "horizontal":
        pass

    if direction == "vertical":
        pass

    if direction == "diagonal":
        pass



    flipped_board = new_bit_board(rules)
    if direction == "horizontal":
        cols = get_cols(rules)
        for i in range(cols):
            mask = board & column_bit_mask(rules, i)
            flipped_board |= (mask >> i) << (cols - 1 - i)
    else:
        rows, cols = get_rows(rules), get_cols(rules)
        for i in range(rows):
            mask = board & row_bit_mask(rules, i)
            flipped_board |= (mask >> i * cols) << ((rows - 1 - i) * cols)
    return flipped_board

In [ ]:
def diagonal_bit_board(board, rules):
    rows, cols = get_rows(rules), get_cols(rules)
    # fix

board = new_bit_board(new_bit_rules())
board |= 1 | 8 | 64
bin(diagonal_bit_board(board))

In [ ]:
def check_row(row, sequence):
    mask = (1 << sequence) - 1
    while row:
        if row < mask: return False
        if (row & mask) == mask: return True
        row >>= 1
    return False

col0 = get_column_mask(0)
col1 = get_column_mask(1)
col2 = get_column_mask(2)
col3 = get_column_mask(3)
col4 = get_column_mask(4)
col5 = get_column_mask(5)
col6 = get_column_mask(6)

row = get_row_mask(5)
bin(row)